In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [19]:
# state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/Arkansas_2018.csv")

# no state or zip code
no_state_no_zip = ['Louisiana', 'Maine', 'South_Dakota', 'Virginia']

# no city or state or zip code
no_city_no_state = ['Minnesota', 'Vermont',]

# no commas
no_commas = ['Arkansas', 'Kentucky', 'Mississippi', 'New_Hampshire', 'New_Jersey']

# already clean
cleaned = ['Alaska', 'Connecticut', 'Delaware', 'Georgia', 'Iowa', 'Illinois', 'Maryland', 'Oklahoma', 'Montana', 'Nebraska', 'North_Carolina',
'Pennsylvania', 'Rhode_Island', 'South_Carolina', 'West_Virginia', 'Wisconsin', "New_Mexico", 'North_Dakota', 'Ohio']

len(cleaned)

19

In [20]:
mass = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/Massachusetts_2018.csv")
def isMA(output):

    '''
    First the first occuracne of Vote and the last occurance of . Remove all text in between that position
    Remove Words before - in the begining 
    e.g. SCHOOL - 165 
    '''

    for i,r in output.iterrows():
        if output.loc[i,'state']=='MA':
            if "vote" in output.loc[i,'address'].lower() or "voter" in output.loc[i,'address'].lower():
                start_pos=output.loc[i,'address'].lower().find("vote")
                end_pos=output.loc[i,'address'].rfind(".")
                address=output.loc[i,'address']

                if end_pos==len(output.loc[i,'address'])-1:

                    address=output.loc[i,'address'][:start_pos]+""+"MA"

                else:
                    address=output.loc[i,'address'][:start_pos]+" "+output.loc[i,'address'][end_pos+2:]

                address=address.replace(".",",")

                if address[-2:]!='MA':
                    address=address+' MA'
                output.loc[i,'address']=address


            if "-" in output.loc[i,'address'].lower():
                address=output.loc[i,'address']

                end_pos=output.loc[i,'address'].lower().find("-")
                address=address[end_pos+1:]

                output.loc[i,'address']=address
    output.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/Massachusetts_2018_clean.csv") 

    return output

new_mass = isMA(mass)
new_mass

,state,jurisdiction_type,county_name,jurisdiction,precinct_id,name,address
0,MA,town,Plymouth,ABINGTON,0-01,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
1,MA,town,Plymouth,ABINGTON,0-02,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
2,MA,town,Plymouth,ABINGTON,0-03,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
3,MA,town,Plymouth,ABINGTON,0-04,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
4,MA,town,Plymouth,ABINGTON,0-05,ABINGTON HIGH SCHOOL,201 GLINIEWICZ WAY
...,...,...,...,...,...,...,...
2169,MA,town,Barnstable,YARMOUTH,0-3,TOWN HALL HEARING ROOM,"1146 RTE.28,S.YARMOUTH"
2170,MA,town,Barnstable,YARMOUTH,0-4,YARMOUTH SENIOR CENTER,"528 FOREST ROAD, SOUTH YARMOUTH"
2171,MA,town,Barnstable,YARMOUTH,0-5,CC&ISLANDS ASSOC REALTORS,"22 MID TECH DRIVE, WEST YARMOUTH"
2172,MA,town,Barnstable,YARMOUTH,0-6,CC&ISLANDS ASSOC REALTORS,"22 MID TECH DRIVE, WEST YARMOUTH"


In [21]:
def add_state(states):
    for state in states:
        #Load state CSV
        state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/{state}_2018.csv")
        state_df.columns = [x.lower() for x in state_df.columns]
        #Remove NA's
        state_df = state_df[state_df['address'].notna()]
        #Remove Duplicates
        state_df = state_df.drop_duplicates(subset=['name','address'], ignore_index=True)
        state_df['address'] = state_df['address'].astype(str) + ", " + state_df["state"]
        state_df.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv") 

    # return state_df
add_state(no_state_no_zip)

In [23]:
def add_city_state(states):
        for state in states:
            #Load state CSV
            state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/{state}_2018.csv")
            #Remove NA's
            state_df = state_df[state_df['address'].notna()]
            #Remove Duplicates
            state_df = state_df.drop_duplicates(subset=['name','address'], ignore_index=True)
            state_df.columns = [x.lower() for x in state_df.columns]
            
            if state == 'Virginia':
                #add city and state with commas 
                state_df['address'] = state_df['address'].astype(str) + ", " + state_df['name'].astype(str) + ', ' + state_df['state'].astype(str)
            else: 
                #add city and state with commas
                state_df['address'] = state_df['address'].astype(str) + ", " + state_df['precinct_name'].astype(str) + ', '+ state_df['state'].astype(str)

            state_df.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv") 
        # return state_df
add_city_state(no_city_no_state)

In [24]:
def already_cleaned(states):
        for state in states:
            #Load state CSV
            state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/{state}_2018.csv")
            state_df.columns = [x.lower() for x in state_df.columns]
            #Remove NA's
            state_df = state_df[state_df['address'].notna()]
            #Remove Duplicates
            state_df = state_df.drop_duplicates(subset=['name','address'], ignore_index=True) 
            state_df.columns = [x.lower() for x in state_df.columns]
            state_df.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv") 
already_cleaned(cleaned)

In [25]:

def add_commas(states):
        for state in states:
            #Load state CSV
            state_df = pd.read_csv(f"../../00_source_data/Polling Data by Year 2018/{state}_2018.csv")
            state_df.columns = [x.lower() for x in state_df.columns]
            #Remove NA's
            state_df = state_df[state_df['address'].notna()]
            #Remove Duplicates
            state_df = state_df.drop_duplicates(subset=['name','address'], ignore_index=True)
            if state == ('New_Hamshire' or 'Mississippi'):
                if isinstance(address[0],str):
                    m = re.search(r"\d", address)
                    address=address[m.start():]

            for i in range(0,len(state_df)):
                #Add street, state, and zip code commas
                start_list = state_df.loc[i,'address'].split()
                #Catch Arkansas addresses that are short
                if (len(start_list) > 3):
                    draft_str2 = ' '.join([str(start_list[i]) for i in range(0,len(start_list) - 3)])
                    new_str = ', '.join([draft_str2, start_list[-3], start_list[-2], start_list[-1]])
                else:
                    new_str = ', '.join(start_list)
                state_df.loc[i,'address'] = new_str
            state_df.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv")

add_commas(no_commas)

In [37]:
import glob
data = pd.DataFrame()
for filename in glob.glob(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/*"):
    df = pd.read_csv(filename)
    data = pd.concat([data, df['address']], ignore_index= True)

# pd.read_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/{state}_2018_clean.csv")

In [41]:
data.columns = ['address']
data
data.to_csv(f"../../20_intermediate_files/10_polling_places/2018_Polling_Clean/cleaned_2018_polling_place.csv")